In [7]:
#http://www.pyimagesearch.com/2014/05/05/building-pokedex-python-opencv-perspective-warping-step-5-6/ # perspetive
import cv2
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import sys
%matplotlib inline

### File location

change these strings to the path of

* camCalibFile    = chessboard img
* white_output    = final output vid path
* nameOfTheFile   = project video path

In [ ]:
camCalibFile = 'D:\\tutorials\\nanodegree\\SDC\\unit_10_Advanced_lane_finding\\CarND-Advanced-Lane-Lines-master\\camera_cal\\calibration*.jpg'
white_output = 'D:\\tutorials\\nanodegree\\SDC\\unit_10_Advanced_lane_finding\\CarND-Advanced-Lane-Lines-master\\FinalOutput10.mp4'
nameOfTheFile = "D:\\tutorials\\nanodegree\\SDC\\unit_10_Advanced_lane_finding\\CarND-Advanced-Lane-Lines-master\\project_video.mp4"


### Cam calibration

steps

* load the chessboard pic
* find corners (9,6)
* create obj point (X,Y,Z) and img points (X,Y)
* use opencv calibrateCamera function to calculate the cam matrix and distorsion coefficient


In [26]:
import glob
image = glob.glob(camCalibFile)
objPoint = []
imgPoint = []
objp = np.zeros((6*9,3),np.float32)
ob=[]
objp[:,:2] =np.mgrid[0:9,0:6].T.reshape(-1,2)


for i in image:
    
    img = cv2.imread(i)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret,corners = cv2.findChessboardCorners(gray,(9,6),None)#8,6
    if ret == True:
        objPoint.append(objp)
        imgPoint.append(corners)
        

img = cv2.imread(image[0])
r , mat, dis ,rvecs, tvecs = cv2.calibrateCamera(objPoint , imgPoint , img.shape[0:2],None,None)





### Line class 
create a class to store the lane lines features like 
* fit polynomial
* fit x values
* radius of curvature
* no of iterations
* current line fit


In [27]:
class Line() :
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False
        #no of iterations
        self.iter = 1
        
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.besx = None # see later 
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None # see later
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') # see later
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
        self.pts = np.array([None])
    

### HLS conversion function

to convert the image frame from RGB to HLS and apply thresholds to it

In [28]:
def HLSconv(img , s_thr_max = 255 , s_thr_min = 100 ) :
    hls = cv2.cvtColor(img,cv2.COLOR_RGB2HLS)
    s_img = hls[:,:,2]
    s_bin = np.zeros_like(s_img)
    s_bin[(s_img>s_thr_min)&(s_img < s_thr_max)] = 1
    return s_bin

### sobel gradient function

to convert the image frame from RGB to grayscape and apply sobel gradient along the x axis with threshold to find lines in the image

In [29]:
def Gradconv(gray , thresh_max_grad = 255 , thresh_min_grad = 10):
    
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0)# for x direction
    sobelx = np.absolute(sobelx)
    sobelx = np.uint8(255*sobelx/np.max(sobelx))
    grad_bin = np.zeros_like(gray)
    grad_bin[(sobelx>thresh_min_grad)&(sobelx <thresh_max_grad)] = 1
    
    return grad_bin

### Region of interest function

Define a Roi mask to remove the unwanted lines from our binary image . this will help in focusing more on lane lines

In [30]:
def roiMask (grad_output):
        roiValue = 1
        roi = np.array([[
            (100 , 720),
            (560 , 486),
            (779 , 486),
            (1200 , 720)
        ]], dtype=np.int32)

        mask = np.zeros_like(grad_output)

        cv2.fillPoly(mask, roi, roiValue)

        grad_output_mask = cv2.bitwise_and(mask,grad_output)
        
        return grad_output_mask

### find the center function

find the offset in meters (center)

offset = (centerX - (distence between two lane lines))/meters per pix in x direction

In [31]:
def centerFind(img):
    
        rightX = np.mean(rightLine.allx[:100])
        leftX = np.mean(leftLine.allx[:100])
        centerX = (img.shape[1]/2)
        
        xm_per_pix = 3.7/700

        ### adjustments sub by left value

        leftX = leftX # ramains same
        rightX = rightX-leftX

        centerX = centerX - leftX

        ### center is off by 
        off = (centerX - (rightX/2))* xm_per_pix
        
        return off

### perspective transformation function

to transform the binary thresholded image into birds eye view so that we can find the lane lines , curvatures and center 

* define a source and destination region
* use getPerspectiveTransform to find the  3 X 3 matrix of a perspective transform
* do the same from destination to src to get the inverse transformation 

In [5]:
def perspectiveTransform(gray):
        img_size = (gray.shape[1],gray.shape[0])
        adjx = 200
        adjy = 0    
        src = np.float32([[760 , 486],[1000 , 700],[220 , 700] , [560 , 486]])
        
        des = np.float32([[img_size[0]-adjx,adjy],[img_size[0]-adjx,img_size[1]-adjy],[adjx,img_size[1]-adjy] ,[adjx,adjy]])
        
        ### warp
        M = cv2.getPerspectiveTransform(src,des)
        Minv = cv2.getPerspectiveTransform(des,src)
        return M , Minv

### AdvLaneFinder function

this function takes in the image frame from a video source and returns a image with lane lines marked 

pipeline of this funcion

* take an image frame
* convert to HLS , seperate saturation channel and apply threshold
* convert img to gray , apply sobel gradient along x axis , apply threshold
* combine both these thresholded output into one
* apply smoothening techniques like erode and dilate to remove noices
* apply perspective transformation to the binary image
* use window search method to find the lane lines (first iteration , use histogram , if we already have a valid lane line , use its points as the starting point for searching)
* mark the lane lines and the area within it
* find the alignment difference between the center of car and the lane center
* return an image with lane markings , thresholded image and center alignment marked


In [33]:
def AdvLaneFinder(img) :
    
    img = cv2.undistort(img, mat, dis, None, mat)               # apply distortion correction to the image 
    try :
        gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)             # conv to grayacape image
        
        s_bin = HLSconv(img)                                    # apply HLS conv and threshold 
        
        grad_bin =  Gradconv(gray)                              # apply sobelx gradient  and threshold 
        

        grad_output = np.zeros_like(gray)
        grad_output[(grad_bin == 1)|(s_bin == 1)] = 1           # use OR operator to add both sobelx and
                                                                #saturation threshold into a single image
        
        grad_output_mask = roiMask (grad_output)                # apply ROI 
        
        
        M , Minv = perspectiveTransform(gray)                   # get the perspective transform matrix for both srd - des and des - src
        
        img_size = (gray.shape[1],gray.shape[0])
        
        warped = cv2.warpPerspective(grad_output_mask , M , img_size ) # wasp the binary image to birds eye view
        
        
        ### apply cleaning to remove noices in the binary image
        kernel = np.ones((8,8),np.uint8)
        dilate = cv2.dilate(warped,kernel,iterations = 1)
        erosion = cv2.erode(dilate,kernel,iterations = 1)
        opening = cv2.morphologyEx(erosion, cv2.MORPH_OPEN, kernel)

        

        binary_warped = np.copy(opening)
        
        histogram = np.sum(binary_warped[int(binary_warped.shape[0]/2):,:], axis=0) # histogram along x axis
        
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255      # create imag with 3 layer depth
        
        midpoint = np.int(histogram.shape[0]/2)                                     # divide histogram into two half
        
        ### if we have a valid fit on lane lines , we can use the x values to fit the windows
        
        
        if(leftLine.allx != None):                                                  # if we have a valid fit
            leftx_base = int(np.mean(leftLine.allx[:100]))
            rightx_base = int(np.mean(rightLine.allx[:100]))
        else:
            
            leftx_base = np.argmax(histogram[:midpoint])
            rightx_base = np.argmax(histogram[midpoint:]) + midpoint

        ### simple window search 
        nwindows = 20
        window_height = np.int(binary_warped.shape[0]/nwindows)       
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        leftx_current = leftx_base
        rightx_current = rightx_base
        margin = 100
        minpix = 50
        left_lane_inds = []
        right_lane_inds = []
        for window in range(nwindows):
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
            cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
            cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
            
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

        
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)

        
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds] 

        # fit a second order polynomial
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)

        ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
                
        avgLX=np.mean(left_fitx)
        avgRX=np.mean(right_fitx)
        
        if (leftLine.besx != None):                                             # save lane fit for future usage in the class
            
            leftLine.besx =(avgLX +leftLine.besx)/leftLine.iter                 # avarage 
            rightLine.besx =(avgRX +rightLine.besx)/rightLine.iter              # avarage
            #polynomial coefficients averaged over the last n iterations
            leftLine.best_fit =(left_fit + leftLine.best_fit)/leftLine.iter
            rightLine.best_fit =(right_fit + rightLine.best_fit)/rightLine.iter

             
        else:
            leftLine.besx = avgLX 
            rightLine.besx = avgRX
            
            leftLine.best_fit = left_fit 
            rightLine.best_fit = right_fit
        

        leftLine.recent_xfitted.append(left_fitx)
        rightLine.recent_xfitted.append(right_fitx)
        
        leftLine.current_fit.append(left_fit)
        rightLine.current_fit.append(right_fit)
        
        ### save the lane pixels into class 
        leftLine.allx = leftx
        leftLine.ally = lefty
        rightLine.allx = rightx
        rightLine.ally = righty
        
        
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

        ### find the curvature of the lane lines 
        y_eval = np.max(ploty)
        left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
        right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])

        ym_per_pix = 30/720                                                       # meters per pixel in y dimension
        xm_per_pix = 3.7/700                                                      # meters per pixel in x dimension

        # Fit new polynomials to x,y in world space
        left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
        right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
        # Calculate the new radii of curvature
        left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
        right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
        
        ### note the lane curvature onto the class
        leftLine.radius_of_curvature = left_curverad
        rightLine.radius_of_curvature = right_curverad
        ###
        
        warp_zero = np.zeros_like(warped).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        pts = np.hstack((pts_left, pts_right))
    
        ###!!! points for using in fill polly
        leftLine.pts = pts_left
        rightLine.pts = pts_right
        
        leftLine.Minv = Minv
        rightLine.Minv = Minv
        
        # Draw the lane onto the warped blank image
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

        # Warp the blank back to original image space using inverse perspective matrix (Minv)
        newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
        # Combine the result with the original image
        result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)#img
        
        
        off = centerFind(img)                                                                # find the center misalignment
        ###!!!
        leftLine.off = off
        rightLine.off = off
        ###!!!
        font = cv2.FONT_HERSHEY_SIMPLEX
        q = cv2.putText(result,str(off)+' M',(900,300), font, 1,(255,255,255),2,cv2.LINE_AA) # add the center to the image
        
        leftLine.detected = True
        rightLine.detected = True
        
        
        
        lane =cv2.resize(binary_warped,(300,200),interpolation = cv2.INTER_AREA)
        lane = lane*255
        
        lane_c = np.dstack((lane, lane, lane))
        
        
        
        result[0:200 , 980:1280] = lane_c                                                     # add the birds eye view to the image
            
        
             
    except  :
            
            
            
            leftLine.detected = False
            rightLine.detected = False
            
            warp_zero = np.zeros_like(img[:,:,1]).astype(np.uint8)
            
            color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
            
            pts = np.hstack((leftLine.pts, rightLine.pts))
            
            cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
            
            newwarp = cv2.warpPerspective(color_warp, leftLine.Minv, (img.shape[1], img.shape[0])) 
        
            result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)#img
            
            font = cv2.FONT_HERSHEY_SIMPLEX
            q = cv2.putText(result,str(leftLine.off)+' M',(900,300), font, 1,(255,255,255),2,cv2.LINE_AA)
        
            
            return result
        
        
    return result

    

In [34]:

leftLine = Line()
rightLine = Line()

clip1 = VideoFileClip(nameOfTheFile)
white_clip = clip1.fl_image(AdvLaneFinder) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video D:\tutorials\nanodegree\SDC\unit_10_Advanced_lane_finding\CarND-Advanced-Lane-Lines-master\FinalOutput10.mp4
[MoviePy] Writing video D:\tutorials\nanodegree\SDC\unit_10_Advanced_lane_finding\CarND-Advanced-Lane-Lines-master\FinalOutput10.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [09:16<00:00,  2.47it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: D:\tutorials\nanodegree\SDC\unit_10_Advanced_lane_finding\CarND-Advanced-Lane-Lines-master\FinalOutput10.mp4 

Wall time: 9min 20s
